In [ ]:
!pip install datasets

In [ ]:
!pip install wandb

In [ ]:
! git lfs install

Git LFS initialized.


In [ ]:
!git clone https://huggingface.co/datasets/eriktks/conll2003

Cloning into 'conll2003'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64 (from 1)
Unpacking objects: 100% (64/64), 17.57 KiB | 749.00 KiB/s, done.


In [ ]:
from datasets import load_dataset
import wandb
from collections import Counter


In [ ]:
ls


conll2003/  sample_data/


In [ ]:
cd conll2003/

/content/conll2003


In [ ]:
ls

conll2003.py  README.md


In [ ]:
ls -lh

total 28K
-rw-r--r-- 1 root root 9.4K Oct 10 00:55 conll2003.py
-rw-r--r-- 1 root root  13K Oct 10 00:55 README.md


In [ ]:
!git lfs pull

In [ ]:
ls -lh

total 28K
-rw-r--r-- 1 root root 9.4K Oct 10 00:55 conll2003.py
-rw-r--r-- 1 root root  13K Oct 10 00:55 README.md


In [ ]:
ls

conll2003.py  README.md


In [ ]:
ls -la


total 44
drwxr-xr-x 3 root root  4096 Oct 10 00:55 ./
drwxr-xr-x 1 root root  4096 Oct 10 00:55 ../
-rw-r--r-- 1 root root  9570 Oct 10 00:55 conll2003.py
drwxr-xr-x 9 root root  4096 Oct 10 01:05 .git/
-rw-r--r-- 1 root root  1174 Oct 10 00:55 .gitattributes
-rw-r--r-- 1 root root 12330 Oct 10 00:55 README.md


In [ ]:
!git lfs ls-files


In [ ]:
!wget -nc https://data.deepai.org/conll2003.zip

--2025-10-10 01:15:14--  https://data.deepai.org/conll2003.zip
Resolving data.deepai.org (data.deepai.org)... 195.181.163.202, 2400:52e0:1a02::975:1
Connecting to data.deepai.org (data.deepai.org)|195.181.163.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 982975 (960K) [application/zip]
Saving to: ‘conll2003.zip’

conll2003.zip       100%[===================>] 959.94K  5.60MB/s    in 0.2s    

2025-10-10 01:15:14 (5.60 MB/s) - ‘conll2003.zip’ saved [982975/982975]



In [ ]:
!unzip -o conll2003.zip -d conll2003_data

Archive:  conll2003.zip
  inflating: conll2003_data/metadata  
  inflating: conll2003_data/test.txt  
  inflating: conll2003_data/train.txt  
  inflating: conll2003_data/valid.txt  


In [ ]:
!ls -lh conll2003_data
# peek at top of train file
!head -n 50 conll2003_data/train.txt

total 4.7M
-rw-r--r-- 1 root root  153 Oct  5  2019 metadata
-rw-r--r-- 1 root root 731K Oct  5  2019 test.txt
-rw-r--r-- 1 root root 3.2M Oct  5  2019 train.txt
-rw-r--r-- 1 root root 809K Oct  5  2019 valid.txt
-DOCSTART- -X- -X- O

EU NNP B-NP B-ORG
rejects VBZ B-VP O
German JJ B-NP B-MISC
call NN I-NP O
to TO B-VP O
boycott VB I-VP O
British JJ B-NP B-MISC
lamb NN I-NP O
. . O O

Peter NNP B-NP B-PER
Blackburn NNP I-NP I-PER

BRUSSELS NNP B-NP B-LOC
1996-08-22 CD I-NP O

The DT B-NP O
European NNP I-NP B-ORG
Commission NNP I-NP I-ORG
said VBD B-VP O
on IN B-PP O
Thursday NNP B-NP O
it PRP B-NP O
disagreed VBD B-VP O
with IN B-PP O
German JJ B-NP B-MISC
advice NN I-NP O
to TO B-PP O
consumers NNS B-NP O
to TO B-VP O
shun VB I-VP O
British JJ B-NP B-MISC
lamb NN I-NP O
until IN B-SBAR O
scientists NNS B-NP O
determine VBP B-VP O
whether IN B-SBAR O
mad JJ B-NP O
cow NN I-NP O
disease NN I-NP O
can MD B-VP O
be VB I-VP O
transmitted VBN I-VP O
to TO B-PP O
sheep NN B-NP O
. . O O

Ger

In [ ]:
ls -lh

total 992K
drwxr-xr-x 2 root root 4.0K Oct 10 01:15 conll2003_data/
-rw-r--r-- 1 root root 9.4K Oct 10 00:55 conll2003.py
-rw-r--r-- 1 root root 960K Dec 18  2022 conll2003.zip
-rw-r--r-- 1 root root  13K Oct 10 00:55 README.md


In [ ]:
cd conll2003_data/

/content/conll2003/conll2003_data


In [ ]:
ls

metadata  test.txt  train.txt  valid.txt


In [ ]:
!head -20 train.txt

-DOCSTART- -X- -X- O

EU NNP B-NP B-ORG
rejects VBZ B-VP O
German JJ B-NP B-MISC
call NN I-NP O
to TO B-VP O
boycott VB I-VP O
British JJ B-NP B-MISC
lamb NN I-NP O
. . O O

Peter NNP B-NP B-PER
Blackburn NNP I-NP I-PER

BRUSSELS NNP B-NP B-LOC
1996-08-22 CD I-NP O

The DT B-NP O
European NNP I-NP B-ORG


In [ ]:
cd ..

/content/conll2003


In [ ]:
import os
from collections import Counter

def read_conll_file(path):
    examples = []
    with open(path, encoding="utf-8") as f:
        tokens, tags = [], []
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    examples.append({"tokens": tokens, "ner_tags": tags})
                    tokens, tags = [], []
            elif not line.startswith("-DOCSTART-"):
                parts = line.split()
                tokens.append(parts[0])
                tags.append(parts[-1])
        if tokens:
            examples.append({"tokens": tokens, "ner_tags": tags})
    return examples

data_dir = "conll2003_data"
train = read_conll_file(os.path.join(data_dir, "train.txt"))
valid = read_conll_file(os.path.join(data_dir, "valid.txt"))
test  = read_conll_file(os.path.join(data_dir, "test.txt"))


**Question1**

In [ ]:
# Count total samples (sentences)
total_samples = len(train) + len(valid) + len(test)

# Count entity distribution
entity_counter = Counter()
for split in [train, valid, test]:
    for ex in split:
        for tag in ex["ner_tags"]:
            if tag != "O":  # ignore non-entities
                # Keep only the entity type (PER, LOC, ORG, MISC)
                entity_type = tag.split("-")[-1]
                entity_counter[entity_type] += 1

print("Total samples:", total_samples)
print("Entity counts:", dict(entity_counter))


Total samples: 20744
Entity counts: {'ORG': 14613, 'MISC': 6779, 'PER': 17050, 'LOC': 12316}


In [ ]:
import wandb

# Initialize W&B project
wandb.init(project="Q1-weak-supervision-ner", name="dataset-summary")

# Log summary metrics
wandb.summary["total_samples"] = total_samples
for entity, count in entity_counter.items():
    wandb.summary[f"entity_count_{entity}"] = count

wandb.log({
    "total_samples": total_samples,
    **{f"entity_count_{k}": v for k, v in entity_counter.items()}
})
wandb.finish()

entity_count_LOC,▁
entity_count_MISC,▁
entity_count_ORG,▁
entity_count_PER,▁
total_samples,▁
entity_count_LOC,12316
entity_count_MISC,6779
entity_count_ORG,14613
entity_count_PER,17050
total_samples,20744


In [ ]:
! pip install snorkel


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
import wandb
from snorkel.labeling import labeling_function, PandasLFApplier, LFAnalysis

In [ ]:
# Flatten train data into tokens
rows = []
for sent in train:  # 'train' from Q1
    for token, tag in zip(sent["tokens"], sent["ner_tags"]):
        rows.append({"token": token, "true_label": tag})

df = pd.DataFrame(rows)
df.head()

,token,true_label
0,EU,B-ORG
1,rejects,O
2,German,B-MISC
3,call,O
4,to,O


In [ ]:
ABSTAIN = -1
DATE = 0   # heuristic: years → DATE/MISC
ORG  = 1   # organization suffix

**Question2**

In [ ]:
@labeling_function()
def lf_detect_year(x):
    if re.search(r"\b(19|20)\d{2}\b", x.token):
        return DATE
    return ABSTAIN

In [ ]:
@labeling_function()
def lf_org_suffix(x):
    if re.search(r"(Inc\.|Corp\.|Ltd\.)$", x.token):
        return ORG
    return ABSTAIN


In [ ]:
def gold_to_label(tag):
    if "ORG" in tag:
        return ORG
    elif "MISC" in tag:
        return DATE
    else:
        return ABSTAIN

df["gold_label"] = df["true_label"].apply(gold_to_label)
import numpy as np

# Convert to numpy array
gold_labels = df["gold_label"].to_numpy()

In [ ]:
lfs = [lf_detect_year, lf_org_suffix]
applier = PandasLFApplier(lfs)
L = applier.apply(df)


100%|██████████| 203621/203621 [00:03<00:00, 54333.95it/s]


In [ ]:
analysis = LFAnalysis(L=L, lfs=lfs).lf_summary(gold_labels)
print(analysis)


                j Polarity  Coverage  Overlaps  Conflicts  Correct  Incorrect  \
lf_detect_year  0      [0]  0.007430       0.0        0.0        4          1   
lf_org_suffix   1      [1]  0.000108       0.0        0.0       22          0   

                Emp. Acc.  
lf_detect_year   0.002644  
lf_org_suffix    1.000000  


In [ ]:
wandb.init(project="Q1-weak-supervision-ner", name="snorkel-lf-summary", reinit=True)

for lf_name in analysis.index:
    wandb.log({
        f"{lf_name}_coverage": analysis.loc[lf_name, "Coverage"],
        f"{lf_name}_accuracy": analysis.loc[lf_name, "Emp. Acc."]
    })

wandb.finish()

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


lf_detect_year_accuracy,▁
lf_detect_year_coverage,▁
lf_org_suffix_accuracy,▁
lf_org_suffix_coverage,▁
lf_detect_year_accuracy,0.00264
lf_detect_year_coverage,0.00743
lf_org_suffix_accuracy,1
lf_org_suffix_coverage,0.00011


**Question3**

In [ ]:
from snorkel.labeling.model import MajorityLabelVoter



In [ ]:
# L is your label matrix from previous step (shape: num_samples x num_LFs)
majority_model = MajorityLabelVoter(cardinality=2)

In [ ]:
# Predict combined label for each token
majority_labels = majority_model.predict(L=L)

In [ ]:
import numpy as np

# Only evaluate non-abstain predictions
mask = majority_labels != -1
accuracy = (majority_labels[mask] == df["gold_label"].to_numpy()[mask]).mean()

print(f"Majority Label Voter Accuracy: {accuracy:.4f}")
print(f"Coverage: {mask.mean():.4f}")  # fraction of tokens labeled


Majority Label Voter Accuracy: 0.0169
Coverage: 0.0075


In [ ]:
wandb.init(project="Q1-weak-supervision-ner", name="majority-voter", reinit=True)

wandb.log({
    "majority_voter_accuracy": accuracy,
    "majority_voter_coverage": mask.mean()
})

wandb.finish()


majority_voter_accuracy,▁
majority_voter_coverage,▁
majority_voter_accuracy,0.01694
majority_voter_coverage,0.00754


In [ ]:
import wandb
import tensorflow as tf
from tensorflow.keras import layers, models

def create_model(num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def train_on_dataset(dataset_name, num_classes, epochs):
    (x_train, y_train), (x_test, y_test) = getattr(tf.keras.datasets, dataset_name).load_data()
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    y_train = tf.keras.utils.to_categorical(y_train, num_classes)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes)

    wandb.init(project="sequential-cifar", name=f"{dataset_name}-run", reinit=True)
    model = create_model(num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Use minimal W&B callback avoiding model graph logging entirely
    class CustomWandbCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            if logs is not None:
                wandb.log(logs)

        def on_train_end(self, logs=None):
            wandb.finish()

    model.fit(x_train, y_train,
              epochs=epochs,
              validation_data=(x_test, y_test),
              callbacks=[CustomWandbCallback()])

    return model




Epoch 1/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 44s 28ms/step - accuracy: 0.0686 - loss: 4.2002 - val_accuracy: 0.1712 - val_loss: 3.4882
Epoch 2/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 40s 26ms/step - accuracy: 0.1961 - loss: 3.3563 - val_accuracy: 0.2265 - val_loss: 3.2031
Epoch 3/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 39s 25ms/step - accuracy: 0.2379 - loss: 3.1125 - val_accuracy: 0.2504 - val_loss: 3.0934
Epoch 4/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 46s 28ms/step - accuracy: 0.2625 - loss: 2.9713 - val_accuracy: 0.2588 - val_loss: 3.0313
Epoch 5/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 40s 26ms/step - accuracy: 0.2815 - loss: 2.8827 - val_accuracy: 0.2571 - val_loss: 3.0214
Epoch 6/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 40s 25ms/step - accuracy: 0.2945 - loss: 2.7995 - val_accuracy: 0.2713 - val_loss: 2.9694
Epoch 7/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 40s 26ms/step - accuracy: 0.3120 - loss: 2.7280 - val_accuracy: 0.2806 - val_loss: 2.9195
Epoch 8/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 40s 25ms/step - accuracy: 

accuracy,▁▁▂▂▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████
loss,█▇▇▇▆▆▆▅▅▅▅▄▄▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
val_accuracy,▁▅▆▇████▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▅▅▄▅▄▄▄▄▄▄▄▄▄
val_loss,▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▇▇▇▇▇███
accuracy,0.70566
loss,0.9626
val_accuracy,0.2259
val_loss,7.05299


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Epoch 1/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 44s 27ms/step - accuracy: 0.4068 - loss: 1.6517 - val_accuracy: 0.5315 - val_loss: 1.3218
Epoch 2/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 43s 28ms/step - accuracy: 0.5836 - loss: 1.1898 - val_accuracy: 0.5928 - val_loss: 1.1465
Epoch 3/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 79s 25ms/step - accuracy: 0.6258 - loss: 1.0685 - val_accuracy: 0.6100 - val_loss: 1.1144
Epoch 4/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 40s 26ms/step - accuracy: 0.6599 - loss: 0.9783 - val_accuracy: 0.6285 - val_loss: 1.0732
Epoch 5/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 40s 26ms/step - accuracy: 0.6756 - loss: 0.9183 - val_accuracy: 0.6284 - val_loss: 1.0763
Epoch 6/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 43s 27ms/step - accuracy: 0.6980 - loss: 0.8653 - val_accuracy: 0.6302 - val_loss: 1.0618
Epoch 7/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 41s 26ms/step - accuracy: 0.7185 - loss: 0.8119 - val_accuracy: 0.6160 - val_loss: 1.1295
Epoch 8/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 41s 26ms/step - accuracy: 

In [ ]:
# Run sequential training experiments

model_100 = train_on_dataset('cifar100', 100, 100)
model_10_from_100 = train_on_dataset('cifar10', 10, 100)

model_10 = train_on_dataset('cifar10', 10, 100)
model_100_from_10 = train_on_dataset('cifar100', 100, 100)